In [1]:
import pandas as pd
import cv2
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

2023-07-26 20:07:08.619768: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-26 20:07:08.621885: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-26 20:07:08.689348: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-26 20:07:08.690930: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-26 20:07:10.449307: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:

csv_file = "archive/bbox.csv"
data = pd.read_csv(csv_file)
images_path = "archive/images/" 
images = []
coordinates = []

In [ ]:
for index, row in data.iterrows():
    image_file = images_path + row['name']
    img = cv2.imread(image_file)
    img = cv2.resize(img, (512, 512)) 
    images.append(img)
    x1 = row['x1'] / img.shape[1]
    x2 = row['x2'] / img.shape[1]
    y1 = row['y1'] / img.shape[0]
    y2 = row['y2'] / img.shape[0]
    coordinates.append([x1, y1, x2, y2])

images = np.array(images)
coordinates = np.array(coordinates)



In [4]:
coordinates[0:2]

array([[3.40820312, 0.91210938, 4.53125   , 3.375     ],
       [1.2890625 , 1.21875   , 4.10546875, 7.59375   ]])

In [5]:
# split train and test data
split_ratio = 0.8
split_idx = int(len(images) * split_ratio)

In [6]:
train_images = images[:split_idx]
train_coordinates = coordinates[:split_idx]
test_images = images[split_idx:]
test_coordinates = coordinates[split_idx:]

In [7]:

# Create CNN Model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(512, 512, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(4))  # Çıktı katmanı, x1, y1, x2, y2 koordinatlarını tahmin edecek 4 düğüm


In [8]:
# Model compile
model.compile(optimizer='adam', loss='mse')  # Mean Squared Error (MSE) 

# Model train
hist = model.fit(train_images, train_coordinates, epochs=30, batch_size=32)


Epoch 1/30
22/22 [==============================] - 79s 4s/step - loss: 3807095.0000
Epoch 2/30
22/22 [==============================] - 77s 3s/step - loss: 13.4544
Epoch 3/30
22/22 [==============================] - 76s 3s/step - loss: 11.3838
Epoch 4/30
22/22 [==============================] - 75s 3s/step - loss: 6.5197
Epoch 5/30
22/22 [==============================] - 77s 3s/step - loss: 3.6918
Epoch 6/30
22/22 [==============================] - 77s 3s/step - loss: 2.3721
Epoch 7/30
22/22 [==============================] - 76s 3s/step - loss: 1.4622
Epoch 8/30
22/22 [==============================] - 76s 3s/step - loss: 1.0324
Epoch 9/30
22/22 [==============================] - 78s 4s/step - loss: 0.7856
Epoch 10/30
22/22 [==============================] - 78s 4s/step - loss: 0.6266
Epoch 11/30
22/22 [==============================] - 77s 4s/step - loss: 0.4716
Epoch 12/30
22/22 [==============================] - 77s 3s/step - loss: 0.4044
Epoch 13/30
22/22 [======================

In [9]:
model.save("model.h5")

/home/berkkucukk/.local/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [10]:

predictions = model.predict(test_images)

result_df = pd.DataFrame({'x1_tahmin': predictions[:, 0], 'y1_tahmin': predictions[:, 1],
                          'x2_tahmin': predictions[:, 2], 'y2_tahmin': predictions[:, 3]})
result_df['x1_gercek'] = test_coordinates[:, 0]
result_df['y1_gercek'] = test_coordinates[:, 1]
result_df['x2_gercek'] = test_coordinates[:, 2]
result_df['y2_gercek'] = test_coordinates[:, 3]

result_df.to_csv('kordinat_tahminleri2.csv', index=False)


6/6 [==============================] - 6s 914ms/step


In [23]:
import cv2
image_path = '/home/berkkucukk/PythonProjects/persondetect2/archive/images/photo-1671726203422-dd710fd8a72d.jpg'
img = cv2.imread(image_path)
img = cv2.resize(img, (512, 512))  
img = np.expand_dims(img, axis=0)  

# Predict
predictions = model.predict(img)
print(predictions[0])
print(predictions)
x1, y1, x2, y2 = predictions[0].astype(float)  
x1 = x1*512
y1 = y1*512
x2 = x2*512
y2 = y2*512
print(x1, y1, x2, y2)
x1 = int(x1)
x2 = int(x2)
y1 = int(y1)
y2 = int(y2)
img2 = cv2.imread(image_path)

cv2.rectangle(img2, (x1, y1), (x2, y2), (0, 255, 0), 2)

cv2.imshow('Kordinat Tahmini', img2)
cv2.waitKey(0)
cv2.destroyAllWindows()

1/1 [==============================] - 0s 53ms/step
[1.6841125 0.7992023 3.4846146 3.1121206]
[[1.6841125 0.7992023 3.4846146 3.1121206]]
862.265625 409.19158935546875 1784.1226806640625 1593.40576171875
